In [95]:
%reload_ext autoreload
%autoreload 2

import cv2
import numpy as np
import os
from aruco import detect_aruco, draw_aruco, select_aruco_poses, PoseSelectors, \
    select_aruco_markers
from segmentation import segment_and_draw_boxes_by_aruco, segment_scene_colorful, \
    segment_red_boxes_hsv, segment_blue_boxes_hsv, segment_goal_hsv, segment_stop_line_hsv
from calibrate_table import calibrate_table
from utils import show, stream_table_frame, stream_segmented_scene, \
    stream_aruco_detected_on_boxes
from aruco_detection_configs import aruco_dict, aruco_detection_params, retry_rejected_params

In [2]:
calib = np.load('phitz/calib.npz')
K = calib['K']
D = calib['D']

In [3]:
table_aruco_size = 0.132
box_aruco_size = 0.0172
box_size = 0.03

In [ ]:
# calibrate table
image_file = "phitz/table_markers/1.jpg"
image = cv2.imread(image_file)
camera2table, _ = calibrate_table(image, K, D, table_aruco_size)
if camera2table is None:
    print("Could not detect table")
else:
    rvec, _ = cv2.Rodrigues(camera2table[0:3, 0:3])
    tvec = camera2table[0:3, 3]
    draw = image.copy()
    cv2.drawFrameAxes(draw, K, D, rvec, tvec, 0.1)
    show(draw)

In [9]:
# detect table aruco
image_file = "phitz/table_markers/1.jpg"
image = cv2.imread(image_file)
arucos = detect_aruco(image, K=K, D=D, aruco_sizes=table_aruco_size, use_generic=True,
    aruco_dict=aruco_dict, params=aruco_detection_params)
arucos = select_aruco_poses(arucos, PoseSelectors.Z_axis_up)
arucos = select_aruco_markers(arucos, lambda id: id < 4)
draw = image.copy()
draw_aruco(draw, arucos, draw_rejected_only=False, draw_ids=False, K=K, D=D)
show(draw)

In [ ]:
# detect box aruco
folder = "phitz/test_aruco/"
image_files = sorted(os.listdir(folder))
bad_image_files = list()
for image_file in image_files:
    image = cv2.imread(folder + image_file)
    arucos = detect_aruco(image, K=K, D=D, aruco_sizes=box_aruco_size, use_generic=True,
        retry_rejected=True, retry_rejected_params=retry_rejected_params,
        aruco_dict=aruco_dict, params=aruco_detection_params)
    arucos = select_aruco_poses(arucos, PoseSelectors.Z_axis_up)
    arucos = select_aruco_markers(arucos, lambda id: id >= 4)
    draw = image.copy()
    draw_aruco(draw, arucos, draw_rejected_only=False, draw_ids=False, K=K, D=D)
    # show(draw)
    # cv2.imwrite("phitz/test_aruco_vis/" + image_file, draw)
    if arucos.n != 2:
        print(f"{image_file}: {arucos.n}")
        bad_image_files.append(image_file)
print(len(bad_image_files))
print(bad_image_files)

In [ ]:
# segment scene
image_file = "phitz/test/0000.jpg"
image = cv2.imread(image_file)
segmentation, (num_red, num_blue) = segment_scene_colorful(image, "front")
print(f"Segmented {num_red} red, {num_blue} blue")
show(segmentation)
# cv2.imwrite("segmentation.jpg", segmentation)

In [136]:
# prepare prompts
for camera in ("front", "top"):
    image_file = f"images/camera/{camera}.jpg"
    image = cv2.imread(image_file)
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV_FULL)
    # show(hsv)
    for prompt in ("red_box", "blue_box"):
        if prompt == "red_box":
            segmentation, _ = segment_red_boxes_hsv(hsv)
        elif prompt == "blue_box":
            segmentation, _ = segment_blue_boxes_hsv(hsv, camera, only_roi=False)
        segmentation *= 255
        # show(segmentation)

        if camera == "front" and prompt == "red_box":
            x = 746
            y = 469
        if camera == "front" and prompt == "blue_box":
            x = 535
            y = 469
        if camera == "top" and prompt == "red_box":
            x = 561
            y = 525
        if camera == "top" and prompt == "blue_box":
            x = 655
            y = 512
        w = 256
        h = 128
        half_w = int(w / 2)
        half_h = int(h / 2)
        segm = segmentation[y - half_h:y + half_h, x - half_w: x + half_w]
        # show(segm)
        if camera == "top":
            segm_to_save = 255 - cv2.rotate(segm, cv2.ROTATE_180)
        else:
            segm_to_save = 255 - segm
        cv2.imwrite(f"images/prompts/segm/{prompt}_{camera}.png", segm_to_save)
        crop = image[y - half_h:y + half_h, x - half_w: x + half_w]
        img = np.ones(crop.shape, dtype=np.uint8) * 255
        mask = segm == 255
        img[mask] = crop[mask]
        if camera == "top":
            img_to_save = cv2.rotate(img, cv2.ROTATE_180)
        else:
            img_to_save = img
        cv2.imwrite(f"images/prompts/img/{prompt}_{camera}.png", img_to_save)

for camera in ("front", "top"):
    image_file = f"images/camera/{camera}.jpg"
    image = cv2.imread(image_file)
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV_FULL)
    # show(hsv)
    for prompt in ("stop_line",):
        if prompt == "stop_line":
            segmentation, _ = segment_stop_line_hsv(hsv, camera, only_roi=False)
        segmentation *= 255
        # show(segmentation)

        if camera == "front" and prompt == "stop_line":
            x = 643
            y = 345
        if camera == "top" and prompt == "stop_line":
            x = 631
            y = 642
        w_scale = 2
        w = 256 * w_scale
        h = 128
        half_w = int(w / 2)
        half_h = int(h / 2)
        segm = segmentation[y - half_h:y + half_h, x - half_w: x + half_w]
        # show(segm)
        segm_embedded = np.zeros((h, int(w / w_scale)), dtype=np.uint8)
        segm_embedded[int(half_h / 2): int(half_h / 2) + half_h, :] = \
            cv2.resize(segm, (int(w / w_scale), int(h / w_scale)), interpolation=cv2.INTER_NEAREST)
        if camera == "top":
            segm_to_save = 255 - cv2.rotate(segm_embedded, cv2.ROTATE_180)
        else:
            segm_to_save = 255 - segm_embedded
        cv2.imwrite(f"images/prompts/segm/{prompt}_{camera}.png", segm_to_save)
        crop = image[y - half_h:y + half_h, x - half_w: x + half_w]
        img = np.ones(crop.shape, dtype=np.uint8) * 255
        mask = segm == 255
        img[mask] = crop[mask]
        img_embedded = np.ones((h, int(w / w_scale), 3), dtype=np.uint8) * 255
        img_embedded[int(half_h / 2): int(half_h / 2) + half_h, :] = \
            cv2.resize(img, (int(w / w_scale), int(h / w_scale)), interpolation=cv2.INTER_NEAREST)
        if camera == "top":
            img_to_save = cv2.rotate(img_embedded, cv2.ROTATE_180)
        else:
            img_to_save = img_embedded
        cv2.imwrite(f"images/prompts/img/{prompt}_{camera}.png", img_to_save)

for camera in ("front", "top"):
    image_file = f"images/camera/{camera}_goal.jpg"
    image = cv2.imread(image_file)
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV_FULL)
    for prompt in ("goal",):
        if prompt == "goal":
            segmentation, _ = segment_goal_hsv(hsv, camera, only_roi=False)
        segmentation *= 255
        # show(segmentation)

        if camera == "front" and prompt == "goal":
            x = 638
            y = 370
        if camera == "top" and prompt == "goal":
            x = 648
            y = 587
        scale = 3
        w = 256 * scale
        h = 128 * scale
        half_w = int(w / 2)
        half_h = int(h / 2)
        segm = segmentation[y - half_h:y + half_h, x - half_w: x + half_w]
        segm_resized = cv2.resize(segm, (int(w / scale), int(h / scale)), interpolation=cv2.INTER_NEAREST)
        # show(segm)
        if camera == "top":
            segm_to_save = 255 - cv2.rotate(segm_resized, cv2.ROTATE_180)
        else:
            segm_to_save = 255 - segm_resized
        cv2.imwrite(f"images/prompts/segm/{prompt}_{camera}.png", segm_to_save)
        crop = image[y - half_h:y + half_h, x - half_w: x + half_w]
        img = np.ones(crop.shape, dtype=np.uint8) * 255
        mask = segm == 255
        img[mask] = crop[mask]
        img_resized = cv2.resize(img, (int(w / scale), int(h / scale)), interpolation=cv2.INTER_NEAREST)
        if camera == "top":
            img_to_save = cv2.rotate(img_resized, cv2.ROTATE_180)
        else:
            img_to_save = img_resized
        cv2.imwrite(f"images/prompts/img/{prompt}_{camera}.png", img_to_save)

In [133]:
# open camera
cam = cv2.VideoCapture(2)
camera = lambda: cam.read()[1]

In [ ]:
# stream table frame
save_folder = None
stream_table_frame(camera, K, D, table_aruco_size, save_folder=save_folder)

In [ ]:
# stream segmented scene
save_folder = None
stream_segmented_scene(camera, "top", save_folder=save_folder)

In [ ]:
# stream aruco detected on boxes
save_folder = None
stream_aruco_detected_on_boxes(camera, K, D, box_aruco_size, save_folder=save_folder)

In [132]:
# close camera
cam.release()